<a href="https://colab.research.google.com/github/AlexKressner/Business_Intelligence/blob/main/Beispiel_ML_Model_Online_Barber_Shop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Veranschaulichung einfaches Zeitreihenprognosemodell am Beispiel Case Study "Online Barber Shop"
Dieses Notebook zeigt, wie man eine Prognose für die Absätze von drei Produkten für die nächsten 2 Monate erstellt. Die zur Verfügung stehenden Absatzdaten reichen von `2020-10` bis `2022-10`. Die Prognose soll für die Monate `2022-11` und `2022-12` erstellt werden.

# 1 Daten laden
Bitte achten Sie hier darauf, dass die Datentypen für die Werte in den Spalten geeignet gewählt sind.

In [53]:
! git clone https://github.com/AlexKressner/Business_Intelligence

fatal: destination path 'Business_Intelligence' already exists and is not an empty directory.


In [54]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics

In [55]:
path = "Business_Intelligence/Daten/Barber/"

In [56]:
data = pd.read_csv(path+"barber_shopify_data.csv", sep=";", parse_dates=["Datum"], decimal=",")

In [57]:
data.dtypes

,0
Datum,datetime64[ns]
Produkt,object
Nettomenge,int64
Stueckpreis_brutto,float64
Bruttoumsatz,float64


In [58]:
data.head(10)

,Datum,Produkt,Nettomenge,Stueckpreis_brutto,Bruttoumsatz
0,2020-05-01,Alaunstein,5,8.99,44.949997
1,2020-05-02,Alaunstein,1,8.99,8.990000
2,2020-05-03,Alaunstein,0,8.99,0.000000
3,2020-05-04,Alaunstein,1,9.99,9.990000
4,2020-05-05,Alaunstein,1,9.98,9.980000
5,2020-05-06,Alaunstein,0,9.98,0.000000
6,2020-05-07,Alaunstein,1,9.99,9.990000
7,2020-05-08,Alaunstein,1,9.99,9.990000
8,2020-05-09,Alaunstein,1,9.99,9.990000
9,2020-05-10,Alaunstein,0,9.99,0.000000


#2 Datenaufbereitung
Wie in der Fallstudie "Regression" werden die Absätze (`Nettomenge`) für jedes Produkt von Tagen auf Monate aggregiert. Dafür leiten wir aus dem Datum das Merkmal `YearMonth` ab und summieren die täglichen Nettomengen bzw. mitteln den täglichen Stückpreis auf Monate.

In [59]:
data["YearMonth"] = data["Datum"].dt.to_period("M")

In [60]:
absatz_monat = data.groupby(["Produkt","YearMonth"]).agg(
    Absatz=("Nettomenge", "sum"),
    Stueckpreis=("Stueckpreis_brutto","mean")
)
absatz_monat.head()

Absatz  Stueckpreis
Produkt    YearMonth                     
Alaunstein 2020-05        25     9.891774
           2020-06        25     9.988333
           2020-07        48     9.988118
           2020-08        75     9.987450
           2020-09        51     9.987694

In [61]:
# mit reset_index() sind Produkt und YearMonth wieder Spalten des DataFrames,
# der nach der Gruppierung resultiert
absatz_monat.reset_index(inplace=True)

In [62]:
absatz_monat.head()

,Produkt,YearMonth,Absatz,Stueckpreis
0,Alaunstein,2020-05,25,9.891774
1,Alaunstein,2020-06,25,9.988333
2,Alaunstein,2020-07,48,9.988118
3,Alaunstein,2020-08,75,9.987450
4,Alaunstein,2020-09,51,9.987694


# 3 Feature Engineering
Es stellt sich die Frage, welche weiteren Merkmale die monatlichen Absätze erklären. Der Monat und das Jahr könnten eine Rolle spielen. Zum Beispiel kann man sich gut vorstellen, dass es absatzstärkere sowie -schwächere Monate gibt. Das Modell sollte also das Feature `Monat` zur Verfügung haben, um solche Zusammenhänge zu erkennen. Auch die Absätze vergangener Monate sind vermutlich gut dafür geeignet zukünftige Absatz vorherzusagen.

In [63]:
# Ableitung einfacher zeitlicher Features
absatz_monat["Monat"] = absatz_monat["YearMonth"].dt.month
absatz_monat["Jahr"] = absatz_monat["YearMonth"].dt.year

In [64]:
# Rekursive Features, d.h. Absätze der Vormonate -1 bis -5 helfen, die zukünftigen Absätze zu prognostizieren
absatz_monat["Absatz_shift_1"] = absatz_monat.groupby("Produkt")["Absatz"].transform(lambda x: x.shift(1))
absatz_monat["Absatz_shift_2"] = absatz_monat.groupby("Produkt")["Absatz"].transform(lambda x: x.shift(2))
absatz_monat["Absatz_shift_3"] = absatz_monat.groupby("Produkt")["Absatz"].transform(lambda x: x.shift(3))
absatz_monat["Absatz_shift_4"] = absatz_monat.groupby("Produkt")["Absatz"].transform(lambda x: x.shift(4))
absatz_monat["Absatz_shift_5"] = absatz_monat.groupby("Produkt")["Absatz"].transform(lambda x: x.shift(5))

In [65]:
absatz_monat[absatz_monat.Produkt=="Alaunstein"].tail()

,Produkt,YearMonth,Absatz,Stueckpreis,Monat,Jahr,Absatz_shift_1,Absatz_shift_2,Absatz_shift_3,Absatz_shift_4,Absatz_shift_5
25,Alaunstein,2022-06,192,9.987538,6,2022,127.0,213.0,129.0,103.0,149.0
26,Alaunstein,2022-07,237,9.987694,7,2022,192.0,127.0,213.0,129.0,103.0
27,Alaunstein,2022-08,177,9.987234,8,2022,237.0,192.0,127.0,213.0,129.0
28,Alaunstein,2022-09,123,9.904115,9,2022,177.0,237.0,192.0,127.0,213.0
29,Alaunstein,2022-10,79,9.687322,10,2022,123.0,177.0,237.0,192.0,127.0


In [66]:
absatz_monat.dropna(inplace=True)

In [67]:
# Was sind die Features!
FEATURES = [
    "Produkt","Monat","Jahr", "Stueckpreis",
    'Absatz_shift_1', 'Absatz_shift_2',
    'Absatz_shift_3', 'Absatz_shift_4',
    'Absatz_shift_5'
    ]

In [68]:
# Was sind die kategorialen Features, d.h. "keine Zahlen"
CAT_FEATURES = ["Produkt", "Monat", "Jahr"]

In [69]:
# Was wollen wir vorhersagen
TARGET = "Absatz"

# 4 Daten vorbereiten und Prognosemodell trainieren und bewerten

In [70]:
# Sortieren der Daten, damit diese je Produkt chronologisch geordnet sind
# YearMonth als Index des DataFrames mitführen
absatz_monat.sort_values(by=["Produkt","YearMonth"], inplace=True)
absatz_monat.set_index("YearMonth", inplace=True)

In [71]:
# Unterscheidung Features und Target
X = absatz_monat[FEATURES]
y = absatz_monat[TARGET]

In [72]:
# Kategoriale Features müssen für die Anwendung des ML-Modells umcodiert werden
X = pd.get_dummies(X, columns=CAT_FEATURES)

In [73]:
X.head()

,Stueckpreis,Absatz_shift_1,Absatz_shift_2,Absatz_shift_3,Absatz_shift_4,Absatz_shift_5,Produkt_Alaunstein,Produkt_Bartbalm Crusoe,Produkt_Rasierer,Monat_1,...,Monat_6,Monat_7,Monat_8,Monat_9,Monat_10,Monat_11,Monat_12,Jahr_2020,Jahr_2021,Jahr_2022
YearMonth,,,,,,,,,,,,,,,,,,,,,
2020-10,9.989677,51.0,75.0,48.0,25.0,25.0,True,False,False,False,...,False,False,False,False,True,False,False,True,False,False
2020-11,9.635083,22.0,51.0,75.0,48.0,25.0,True,False,False,False,...,False,False,False,False,False,True,False,True,False,False
2020-12,9.989440,108.0,22.0,51.0,75.0,48.0,True,False,False,False,...,False,False,False,False,False,False,True,True,False,False
2021-01,9.987769,109.0,108.0,22.0,51.0,75.0,True,False,False,True,...,False,False,False,False,False,False,False,False,True,False
2021-02,9.987747,54.0,109.0,108.0,22.0,51.0,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [74]:
# Aufteilung der Zeitreihendaten in Trainingsdaten (<=split_year_month) und Testdaten (>split_year_month)
split_year_month = "2022-08"

In [75]:
# Aufteilen der Daten
X_train = X[X.index<=split_year_month]
y_train = y[y.index<=split_year_month]
X_test = X[X.index>split_year_month]
y_test = y[y.index>split_year_month]

In [76]:
# Modell auswählen
model = RandomForestRegressor()

In [77]:
# Modell trainieren
model.fit(X_train, y_train)

RandomForestRegressor()

In [78]:
# Modell anwenden
y_pred = model.predict(X_test)

In [79]:
# Modell bewerten
sklearn.metrics.mean_absolute_error(y_test, y_pred)

99.61833333333334

# 5 Prognosemodell für Vorhersage verwenden

In [80]:
# Wir trainieren nun das Modell auf allen Daten, d.h. von "2020-10" bis "2022-10"
# Zuvor hatten wir für das Training nur die Daten von "2020-10" bis "2022-08" verwendet
model.fit(X, y)

RandomForestRegressor()

**WICHTIG**: Wir müssen nun einen Datensatz erzeugen, den wir in das Prognosemodell geben, um eine Vorhersage für die Monate `2022-11` und `2022-12` für jedes der Produkte zu erhalten!

In [81]:
# Prognosemonate
pred_year_month = ["2022-11","2022-12"]

In [82]:
# Produkte, für die eine Prognose gemacht werden soll
products = absatz_monat["Produkt"].unique()

In [83]:
# Preis-Feature: Wir nehmen an, die Werte hat die Vertriebsabteilung geplant.
planned_prices = {
    "Alaunstein": 9.9,
    "Bartbalm Crusoe": 20,
    "Rasierer": 35
}

In [84]:
# Wir müssen jetzt einen Datensatz erzeugen, der wie X_test und X_train strukturiert ist!
new_data = []

# Iteration über Monate, für die die Absätze vorhergesagt werden sollen
# und das jeweilige Produkt
for ym in pred_year_month:
  for prod in products:
    # Erstellen neuer Reihen für den DataFrame, auf den dann das Prognosemodell angewendet werden soll
    row = {}
    row["YearMonth"] = pd.Period(ym, freq='M')
    row["Produkt"] = prod
    row["Monat"] = pd.Period(ym, freq='M').month
    row["Jahr"] = pd.Period(ym, freq='M').year
    row["Stueckpreis"] = planned_prices.get(prod)  # geplante Preise

    # die letzten 5 Monate von Absätzen - dieses Feature wollen wir bei der Prognose nutzen
    last_sales = absatz_monat[absatz_monat["Produkt"] == prod].tail(5)
    for i in range(1, 6):
      row[f"Absatz_shift_{i}"] = last_sales["Absatz"].shift(i).iloc[-1]

    # Neue Reihe speichern
    new_data.append(row)

In [85]:
# Daten in DataFrame umwandeln
pred_data = pd.DataFrame(new_data)

In [86]:
pred_data.head(10)

,YearMonth,Produkt,Monat,Jahr,Stueckpreis,Absatz_shift_1,Absatz_shift_2,Absatz_shift_3,Absatz_shift_4,Absatz_shift_5
0,2022-11,Alaunstein,11,2022,9.9,123.0,177.0,237.0,192.0,NaN
1,2022-11,Bartbalm Crusoe,11,2022,20.0,16.0,34.0,36.0,26.0,NaN
2,2022-11,Rasierer,11,2022,35.0,182.0,221.0,198.0,507.0,NaN
3,2022-12,Alaunstein,12,2022,9.9,123.0,177.0,237.0,192.0,NaN
4,2022-12,Bartbalm Crusoe,12,2022,20.0,16.0,34.0,36.0,26.0,NaN
5,2022-12,Rasierer,12,2022,35.0,182.0,221.0,198.0,507.0,NaN


In [87]:
# Neuen DataFrame erzeugen
prediction_df = pred_data[["Produkt","YearMonth"]].copy()

In [88]:
# Kategoriale Features umcodieren
pred_data = pd.get_dummies(pred_data, columns=CAT_FEATURES)

In [89]:
pred_data.head(10)

,YearMonth,Stueckpreis,Absatz_shift_1,Absatz_shift_2,Absatz_shift_3,Absatz_shift_4,Absatz_shift_5,Produkt_Alaunstein,Produkt_Bartbalm Crusoe,Produkt_Rasierer,Monat_11,Monat_12,Jahr_2022
0,2022-11,9.9,123.0,177.0,237.0,192.0,NaN,True,False,False,True,False,True
1,2022-11,20.0,16.0,34.0,36.0,26.0,NaN,False,True,False,True,False,True
2,2022-11,35.0,182.0,221.0,198.0,507.0,NaN,False,False,True,True,False,True
3,2022-12,9.9,123.0,177.0,237.0,192.0,NaN,True,False,False,False,True,True
4,2022-12,20.0,16.0,34.0,36.0,26.0,NaN,False,True,False,False,True,True
5,2022-12,35.0,182.0,221.0,198.0,507.0,NaN,False,False,True,False,True,True


In [90]:
# Spalten des zukünftigen DataFrame an den Trainingsdaten ausrichten
pred_data = pred_data.reindex(columns=X_train.columns, fill_value=0)

In [91]:
pred_data.head(10)

,Stueckpreis,Absatz_shift_1,Absatz_shift_2,Absatz_shift_3,Absatz_shift_4,Absatz_shift_5,Produkt_Alaunstein,Produkt_Bartbalm Crusoe,Produkt_Rasierer,Monat_1,...,Monat_6,Monat_7,Monat_8,Monat_9,Monat_10,Monat_11,Monat_12,Jahr_2020,Jahr_2021,Jahr_2022
0,9.9,123.0,177.0,237.0,192.0,NaN,True,False,False,0,...,0,0,0,0,0,True,False,0,0,True
1,20.0,16.0,34.0,36.0,26.0,NaN,False,True,False,0,...,0,0,0,0,0,True,False,0,0,True
2,35.0,182.0,221.0,198.0,507.0,NaN,False,False,True,0,...,0,0,0,0,0,True,False,0,0,True
3,9.9,123.0,177.0,237.0,192.0,NaN,True,False,False,0,...,0,0,0,0,0,False,True,0,0,True
4,20.0,16.0,34.0,36.0,26.0,NaN,False,True,False,0,...,0,0,0,0,0,False,True,0,0,True
5,35.0,182.0,221.0,198.0,507.0,NaN,False,False,True,0,...,0,0,0,0,0,False,True,0,0,True


In [92]:
# Prognosemodell anwenden
predictions = model.predict(pred_data)

# Ergebnisse eintragen
prediction_df["PredictedDemand"] = predictions

prediction_df.head(10)

,Produkt,YearMonth,PredictedDemand
0,Alaunstein,2022-11,142.86
1,Bartbalm Crusoe,2022-11,48.84
2,Rasierer,2022-11,217.84
3,Alaunstein,2022-12,121.41
4,Bartbalm Crusoe,2022-12,44.57
5,Rasierer,2022-12,167.60
